# 고도몰 SMS 발송내역 분석

8~10월 월별 SMS 발신내역 통계 확인
- 고도몰어드민 SMS 발송 내역보기: http://gdadmin.planbio.godomall.com/member/sms_log.php
- 고도몰 DB 정보: http://doc.godomall5.godomall.com/godo/database/table_layout.php
  - es_smsSendLog, es_smsSendList 테이블 확인


In [1]:
import json
import pandas as pd
import plotly.express as px

df_raw_log = pd.read_excel("data/es_smsLog.xlsx")
df_raw_list = pd.read_excel("data/es_smsSendList.xlsx")

## smsLog 데이터 클랜징

In [2]:
log_columns = [
    "sno",
    "sendFl",
    "smsType",
    "sender",
    "contents",
    "receiverCnt",
    "receiverType",
    "sendStatus",
    "sendSuccessCnt",
    "sendFailCnt",
    "sendDt",
    "smsSendKey",
]
df_log = df_raw_log.copy()[log_columns]
# Convert columns to appropriate data types
df_log["sno"] = df_log["sno"].astype(int)
df_log["sendFl"] = df_log["sendFl"].astype("category")
df_log["smsType"] = df_log["smsType"].astype("category")
df_log["sender"] = df_log["sender"].astype(str)
df_log["contents"] = df_log["contents"].astype(str)
df_log["receiverCnt"] = df_log["receiverCnt"].astype(int)
df_log["receiverType"] = df_log["receiverType"].astype("category")
df_log["sendStatus"] = df_log["sendStatus"].astype("category")
df_log["sendSuccessCnt"] = df_log["sendSuccessCnt"].astype(int)
df_log["sendFailCnt"] = df_log["sendFailCnt"].astype(int)
df_log["sendDt"] = pd.to_datetime(df_log["sendDt"], errors="coerce")
df_log["smsSendKey"] = df_log["smsSendKey"].astype(int)

# 카카오 알림톡 필터링
df_log = df_log[df_log["sendFl"] != "kakao"]


# sender 데이터 다듬기
def extract_sender(string):
    arr = json.loads(string.replace("\\", ""))
    return arr[1]


df_log["sender"] = df_log["sender"].apply(extract_sender)

## smsList 데이터 클랜징

In [3]:
list_columns = [
    "sno",
    "smsLogSno",
    "receiverName",
    "receiverCellPhone",
    "sendCheckFl",
    "acceptCheckFl",
    "regDt",
    "kakaoSendKey",
]
df_list = df_raw_list.copy()[list_columns]
# Convert columns to appropriate data types
df_list["sno"] = df_list["sno"].astype(int)
df_list["smsLogSno"] = df_list["smsLogSno"].astype(int)
df_list["receiverName"] = df_list["receiverName"].astype(str)
df_list["receiverCellPhone"] = df_list["receiverCellPhone"].astype(str)
df_list["sendCheckFl"] = df_list["sendCheckFl"].astype("category")
df_list["acceptCheckFl"] = df_list["acceptCheckFl"].astype("category")
df_list["regDt"] = pd.to_datetime(df_list["regDt"], errors="coerce")
df_list["kakaoSendKey"] = df_list["kakaoSendKey"].astype(float)

# 카카오 알림톡 필터링
df_list = df_list[df_list.kakaoSendKey.isna()]
df_list = df_list.drop("kakaoSendKey", axis=1)

## 통계를 위한 분석 데이터 만들기
- list 데이터에 log의 `["sno", "sendFl", "smsType", "sender", "receiverType"]` 컬럼데이터 추가

In [4]:
# Merge df_list with df_log to add sendFl, smsType, sender, receiverType
df_merged = df_list.merge(
    df_log[["sno", "sendFl", "smsType", "sender", "receiverType"]],
    left_on="smsLogSno",
    right_on="sno",
    how="left",
)

# Drop the redundant 'sno' column from df_log
df_merged = df_merged.drop(columns=["sno_y"])

# Rename columns to avoid confusion
df_merged = df_merged.rename(columns={"sno_x": "sno"})

df_merged = df_merged.dropna()

df_merged

,sno,smsLogSno,receiverName,receiverCellPhone,sendCheckFl,acceptCheckFl,regDt,sendFl,smsType,sender,receiverType
0,430691,75770,이남헌,1083800903,y,y,2024-08-01 00:28:45,sms,order,SYSTEM,group
1,430692,75770,구매물류관리자,1099361333,y,y,2024-08-01 00:28:45,sms,order,SYSTEM,group
2,430693,75770,구매물류관리자,1028485625,y,y,2024-08-01 00:28:45,sms,order,SYSTEM,group
3,430694,75770,박수진,1037191851,y,y,2024-08-01 00:28:45,sms,order,SYSTEM,group
4,430695,75770,이동석,1097888457,y,y,2024-08-01 00:28:45,sms,order,SYSTEM,group
...,...,...,...,...,...,...,...,...,...,...,...
78872,509605,90305,김승재,1065472231,r,y,2024-11-11 16:40:01,sms,order,SYSTEM,group
78873,509606,90305,고동민,1076161680,r,y,2024-11-11 16:40:01,sms,order,SYSTEM,group
78874,509607,90305,박진혁,1043889221,r,y,2024-11-11 16:40:01,sms,order,SYSTEM,group
78875,509608,90305,손민지,1028463252,r,y,2024-11-11 16:40:01,sms,order,SYSTEM,group


## 월별 SMS 전송통계 시각화 함수 정의

In [5]:
def draw_month_counts(category, df_merged):
    # Extract month from regDt
    df_merged["month"] = df_merged["regDt"].dt.month

    # Filter for August, September, and October
    df_filtered = df_merged[df_merged["month"].isin([8, 9, 10])]

    # Group by month and category, then count occurrences
    month_counts = df_filtered.groupby(["month", category]).size().unstack(fill_value=0)

    # Map month numbers to month names
    month_counts.index = month_counts.index.map({8: "8월", 9: "9월", 10: "10월"})
    display(month_counts)

    # Plot the data using plotly
    fig = px.bar(
        month_counts,
        x=month_counts.index,
        y=month_counts.columns,
        title=f"Monthly {category} Counts",
        labels={"value": "Count", "month": "Month"},
        barmode="stack",
    )

    fig.show()

## sendFl(lms, sms) 별 월별 전송 통계

In [6]:
draw_month_counts("sendFl", df_merged)

C:\Users\ybnoh\AppData\Local\Temp\ipykernel_9784\3290971921.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  month_counts = df_filtered.groupby(["month", category]).size().unstack(fill_value=0)


sendFl,kakao,lms,sms
month,,,
8월,0,2448,19470
9월,0,3494,20089
10월,0,3274,20331


## smsType(order, member, board, user) 별 월별 전송 통계

In [7]:
draw_month_counts("smsType", df_merged)

C:\Users\ybnoh\AppData\Local\Temp\ipykernel_9784\3290971921.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



smsType,board,member,order,user
month,,,,
8월,184,2208,19500,26
9월,152,2299,21106,26
10월,203,2419,20958,25


## sender(발송자) 별 전송 통계

In [8]:
draw_month_counts("sender", df_merged)

sender,SYSTEM,planbio_apply,planbio_sales1,plandocs_al,plandocs_cs1,plandocs_cs2,plandocs_cx,plandocs_cy,plandocs_de,plandocs_it2,...,plandocs_md,plandocs_md_1,plandocs_md_2,plandocs_pjh,plandocs_sales2,plandocs_sales3,plandocs_scm1,plandocs_sl,plandocs_ss,sofnt
month,,,,,,,,,,,,,,,,,,,,,
8월,20716,436,2,1,693,2,3,0,5,0,...,18,0,1,1,1,0,1,3,6,1
9월,22060,585,2,1,710,0,34,0,66,6,...,14,1,8,1,0,1,0,7,3,0
10월,22223,863,1,0,0,0,359,6,72,0,...,29,1,0,8,0,0,0,6,11,0


## receiverType(each:회원, group:본사운영자) 별 발송 통계

In [9]:
draw_month_counts("receiverType", df_merged)

C:\Users\ybnoh\AppData\Local\Temp\ipykernel_9784\3290971921.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



receiverType,each,group
month,,
8월,1864,20054
9월,2240,21343
10월,2138,21467
